In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt
import datetime as dt

today = dt.date.today()
today_month = today.month
if (today_month > 7):
    current_year = today.year + 1
else:
    current_year = today.year

In [6]:
def name_standardization_df(df):
    df['Player'] = df.Player.apply(unidecode.unidecode)
    df['Player'] = df.Player.str.replace('.', '')
    df['Player'] = df.Player.str.replace("'", '')
    df['Player'] = df.Player.str.lower()
    df['Player'] = df.Player.str.strip('*')
    return df
def name_standardization_player(player):
    player = unidecode.unidecode(player)
    player = player.replace('.', '')
    player = player.replace("'", '')
    player = player.lower()
    player = player.strip('*')
    return player

In [14]:
# Retreiving injuries
# Getting tables
tables = pd.read_html('https://www.espn.com/nba/injuries')

# Iterating through injury tables to create injury list
injuries = pd.DataFrame(columns = ['NAME', 'STATUS'])
for table in tables:
    table = table[['NAME', 'STATUS']]
    injuries = injuries.append(table)
injuries.reset_index(drop = True, inplace = True)

# Filtering out players who may not be out
injuries = injuries[injuries.STATUS == 'Out']

# Adjusting columns
injuries = injuries[['NAME']]
injuries.columns = ['Player']
injuries = name_standardization_df(injuries)

In [18]:
# Active roster function

# Instantiating necessary items
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.lineups.com/nba/depth-charts')
driver.refresh()
time.sleep(5)
html = driver.page_source
tables = pd.read_html(html)
teams = ['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets',
        'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets',
        'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers',
        'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat',
        'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans',
        'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers',
        'Phoenix Suns', 'Portland Trailblazers', 'Sacramento Kings', 'San Antonio Spurs',
        'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']
def name_adjustment(x):
    try:
        names = x.split(' ')
        if len(names) == 4:
            name = names[0] + ' ' + names[1]
        if len(names) == 6:
            name = names[0] + ' ' + names[1] + ' ' + names[2]
        if len(names) == 5:
            name = names[0] + ' ' + names[1]
        if len(names) == 7:
            name = names[0] + ' ' + names[1] + ' ' + names[2]
    except:
        name = x
    return name

# Getting active rosters into dictionary of lists for each team
team_dict = {}
for team, table in zip(teams,tables):
    table.columns = table.columns.droplevel(0)
    for i in [1,2,3]:
        table[str(i)] = table[str(i)].apply(name_adjustment)
    table = table[['1', '2', '3']]
    players = list()
    for i in range(table.shape[0]): 
        for j in range(table.shape[1]):
            player = table.iloc[i, j]
            if type(player) == float:
                continue
            player_standard = name_standardization_player(player)
            if (player_standard in injuries.Player.unique()):
                pass
            else:
                players = players + [player]
    team_dict[team] = players



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


In [19]:
team_dict

{'Atlanta Hawks': ['Trae Young',
  'Delon Wright',
  'Sharife Cooper',
  'Bogdan Bogdanovic',
  'Lou Williams',
  'Timothe Luwawu-Cabarrot',
  "De'Andre Hunter",
  'Kevin Huerter',
  'Kevin Knox II',
  'John Collins',
  'Jalen Johnson',
  'Clint Capela',
  'Onyeka Okongwu',
  'Gorgui Dieng'],
 'Boston Celtics': ['Marcus Smart',
  'Romeo Langford',
  'Payton Pritchard',
  'Jaylen Brown',
  'Dennis Schroder',
  'Brodric Thomas',
  'Jayson Tatum',
  'Josh Richardson',
  'Aaron Nesmith',
  'Robert Williams III',
  'Grant Williams',
  'Sam Hauser',
  'Al Horford',
  'Enes Freedom',
  'Bruno Fernando'],
 'Brooklyn Nets': ['Kyrie Irving',
  'Patty Mills',
  'Jevon Carter',
  'Cam Thomas',
  'David Duke Jr.',
  'Kessler Edwards',
  'Bruce Brown',
  'Nicolas Claxton',
  'James Johnson',
  "Day'Ron Sharpe",
  "DeAndre' Bembry",
  'Blake Griffin'],
 'Charlotte Hornets': ['LaMelo Ball',
  'James Bouknight',
  'Ish Smith',
  'Terry Rozier',
  'Cody Martin',
  'Scottie Lewis',
  'Kelly Oubre Jr.',
 